In [1]:
import pandas as pd
import numpy as np
from xml.dom.minidom import parse, parseString

In [2]:
def get_data(input_gpx):
    # TODO: you may use your code from exercise 3 here.
    # create function to extract cols 
    def element_to_data(ele):
        lat = float(ele.getAttribute("lat"))
        lon = float(ele.getAttribute("lon"))
        dtime = ele.getElementsByTagName('time')
        dtime = pd.to_datetime(dtime[0].firstChild.nodeValue, utc=True)

        return lat, lon, dtime

    # read file
    gps_doc = parse(input_gpx)
    # get 'trkpt' element from file 
    gps_elements = gps_doc.getElementsByTagName('trkpt')
    # get data from elment and create df with cols 
    gps_data = pd.DataFrame(list(map(element_to_data, gps_elements)),
                             columns=['lat', 'lon', 'timestamp'])
    return gps_data


In [3]:
gps = get_data('walk1/gopro.gpx')
accl = pd.read_json('walk1/accl.ndjson.gz', lines=True, convert_dates=['timestamp'])[['timestamp', 'x']]

phone = pd.read_csv('walk1/phone.csv.gz')[['time', 'gFx', 'Bx', 'By']]
first_time = accl['timestamp'].min()
#phone['timestamp'] = first_time + pd.to_timedelta(phone['time'], unit='sec')


In [4]:
# unify the times, aggregate using 4 second bins 
accl['dt'] = accl['timestamp'].round("4S")
gps['dt'] = gps['timestamp'].round("4S")
#phone['dt'] = phone['timestamp'].round("4S")

In [5]:
# group on the rounded-times, and average all of the other values 
accl = accl.groupby(['dt']).mean()
gps = gps.groupby(['dt']).mean()
#phone = phone.groupby(['dt']).mean()

In [6]:
accl

,x
dt,
2022-06-08 18:12:08+00:00,0.008074
2022-06-08 18:12:12+00:00,-0.241065
2022-06-08 18:12:16+00:00,0.401996
2022-06-08 18:12:20+00:00,0.717109
2022-06-08 18:12:24+00:00,0.889633
...,...
2022-06-08 18:19:08+00:00,0.502401
2022-06-08 18:19:12+00:00,0.705140
2022-06-08 18:19:16+00:00,0.644120


In [7]:
gps

,lat,lon
dt,,
2022-06-08 18:12:08+00:00,49.278633,-123.016149
2022-06-08 18:12:12+00:00,49.278659,-123.016139
2022-06-08 18:12:16+00:00,49.278659,-123.016095
2022-06-08 18:12:20+00:00,49.278638,-123.016024
2022-06-08 18:12:24+00:00,49.278600,-123.015985
...,...,...
2022-06-08 18:19:08+00:00,49.277550,-123.016160
2022-06-08 18:19:12+00:00,49.277542,-123.016076
2022-06-08 18:19:16+00:00,49.277552,-123.015960


In [8]:
phone

,time,gFx,Bx,By
0,0.027121,0.0000,0.0000,0.0000
1,0.098611,0.0000,0.0000,0.0000
2,0.100053,0.0000,-19.0125,4.8000
3,0.101865,-0.0159,-19.0125,4.8000
4,0.146639,-0.0159,-19.0125,4.8000
...,...,...,...,...
38539,419.950411,-0.0845,-16.3313,0.6188
38540,419.978822,-0.0845,-16.3313,0.6188
38541,419.981671,-0.0840,-16.3313,0.6188
38542,419.983144,-0.0840,-16.3313,0.6188


In [9]:
#phone['dt'] = first_time + pd.to_timedelta(phone['time'] + 3.1, unit='sec')
#phone['dt'] = phone['dt'].round("4S")


In [10]:
#phone = phone.groupby(['dt']).aggregate('mean')

In [11]:
phone

,time,gFx,Bx,By
0,0.027121,0.0000,0.0000,0.0000
1,0.098611,0.0000,0.0000,0.0000
2,0.100053,0.0000,-19.0125,4.8000
3,0.101865,-0.0159,-19.0125,4.8000
4,0.146639,-0.0159,-19.0125,4.8000
...,...,...,...,...
38539,419.950411,-0.0845,-16.3313,0.6188
38540,419.978822,-0.0845,-16.3313,0.6188
38541,419.981671,-0.0840,-16.3313,0.6188
38542,419.983144,-0.0840,-16.3313,0.6188


In [12]:
x = phone.join(accl)


In [13]:
# find the offset with the highest cross-correlation 
m = 0
max_index = 0
for offset in np.linspace(-5.0, 5.0, 101):
    p_data = phone
    p_data['dt'] = first_time + pd.to_timedelta(p_data['time'] + offset, unit='sec')
    p_data['dt'] = p_data['dt'].round("4S")
    p_data = p_data.groupby(['dt']).aggregate('mean')
    data = p_data.join(accl).dropna()
    cor_rel = data['gFx'].dot(data['x'])
    if cor_rel >= m:
        m = cor_rel
        max_index = offset

time_diff =  max_index.round(2)       





In [20]:
time_diff
    

-0.6

In [21]:
# remake phone['time'] as phone['time'] + offset
phone = pd.read_csv('walk1/phone.csv.gz')[['time', 'gFx', 'Bx', 'By']]
phone['time'] = phone['time'] + time_diff
phone['timestamp'] = first_time + pd.to_timedelta(phone['time'], unit='sec')
phone['dt'] = phone['timestamp'].round("4S")
phone = phone.groupby(['dt']).mean()

In [22]:
phone

,time,gFx,Bx,By
dt,,,,
2022-06-08 18:12:08+00:00,-0.165579,-0.029419,-18.438694,3.844544
2022-06-08 18:12:12+00:00,2.183039,-0.037484,-18.379600,5.868621
2022-06-08 18:12:16+00:00,6.147850,0.036395,-19.161709,-0.792787
2022-06-08 18:12:20+00:00,10.159373,0.068016,-13.347370,-10.022273
2022-06-08 18:12:24+00:00,14.154097,0.072545,-15.571638,-10.951525
...,...,...,...,...
2022-06-08 18:18:52+00:00,402.146678,0.018038,-20.034918,3.702991
2022-06-08 18:18:56+00:00,406.138197,0.076283,-2.271668,16.942237
2022-06-08 18:19:00+00:00,410.147971,0.124758,-0.647815,15.428634


In [23]:
# combine three dfs
combined = phone.join(accl)
combined = combined.join(gps)
combined = combined.reset_index()

In [24]:
combined[['lat', 'lon', 'dt']]


,lat,lon,dt
0,49.278633,-123.016149,2022-06-08 18:12:08+00:00
1,49.278659,-123.016139,2022-06-08 18:12:12+00:00
2,49.278659,-123.016095,2022-06-08 18:12:16+00:00
3,49.278638,-123.016024,2022-06-08 18:12:20+00:00
4,49.278600,-123.015985,2022-06-08 18:12:24+00:00
...,...,...,...
101,49.277412,-123.016344,2022-06-08 18:18:52+00:00
102,49.277431,-123.016238,2022-06-08 18:18:56+00:00
103,49.277485,-123.016246,2022-06-08 18:19:00+00:00
104,49.277538,-123.016229,2022-06-08 18:19:04+00:00
